<a href="https://colab.research.google.com/github/Luciano1120/Python_L/blob/main/PreEntregaProyectoFinal_LucianoScorza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [6]:

#Pedidos-Cabecera. 
urlP='https://onedrive.live.com/download?resid=68e1f46f4ff0ff93%212767&authkey=AAFAlkQl8kr00BQ&em=2&em=Z&app=Excel'
df=pd.read_pickle(urlP)

#Target
urlT='https://onedrive.live.com/download?resid=68e1f46f4ff0ff93%212768&authkey=AIfPd4wybcORgro&em=2&em=Z&app=Excel'
dfT=pd.read_excel(urlT)

#Sumar otros Dataset para mayor Info- Detalle Item, etc

,PK,Perdida
0,1CA1376,no
1,1CA1377,no
2,1CA1378,no
3,1CA1380,no
4,1CA1385,no
...,...,...
36291,2VB5094,no
36292,2VB5095,no
36293,2VB5096,no
36294,2VB5097,no
